In [42]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [6]:
news_rss = requests.get('https://www.mk.co.kr/rss/50200011/')
news_rss

<Response [200]>

In [ ]:
news_rss.content
news_rss.text

In [9]:
news_rss_soup = BeautifulSoup(news_rss.content,'xml')

In [ ]:
title_list = news_rss_soup.select('item > title')
title_list = [title.text for title in title_list]
title_list

In [ ]:
link_list = news_rss_soup.select('item > link')
link_list = [link.text for link in link_list]
link_list

In [ ]:
news_data = []
news_datetime = []
for link in link_list :
    news_res = requests.get(link)
    news_content_soup = BeautifulSoup(news_res.content, 'lxml')
    news_content = news_content_soup.select_one('.news_cnt_detail_wrap')
    news_data.append(news_content.text)
    news_datetime.append(news_content_soup.select_one('.time_area .registration dd').text)

print(news_data)
print(news_datetime)

In [31]:
len(news_data)

50

In [41]:
len(news_datetime)

50

In [32]:
type(news_data)

list

In [ ]:
[item for item in news_data if re.search(r'\S',item)]

In [48]:
#### 일시, 제목, 내용, 
news_df = pd.DataFrame({
    '일시' : news_datetime,
    '제목' : title_list,
    '내용' : news_data
})


In [49]:
news_df.head()

,일시,제목,내용
0,2024-12-26 09:24:12,"바이오니아 자회사 에이스바이옴, 中 프리미엄 해외 직구 플랫폼 티몰 입점…현지 시장...",\n 사진 확대 [사진제공 = 바이오니아] 바이오니아 자회사 에이스...
1,2024-12-26 08:30:34,"“LG이노텍, 내년 아이폰 출시 전까지 숨고르기 돌입”…목표가↓",\n 사진 확대 LG이노텍 로고. [사진 출처 = LG이노텍] 대신...
2,2024-12-26 08:29:34,"“씨앤씨인터내셔널, 해외 고객사 다변화에 초점…내년 성과 나타날 전망”",\n 사진 확대 씨앤씨인터내셔널 CI. [사진 = 씨앤씨인터내셔널] ...
3,2024-12-26 06:42:22,"“동학개미들은 죽 쑤는데”…외국인 올 코스피 수익률 11%, 뭘 샀길래",\n 사진 확대 올해 순매수 톱10 종목 분석외국인 전력기기·바이...
4,2024-12-26 06:00:00,2025년 자산시장 전략은 본연의 맛 싱글몰트 [매일 돈이 보이는 습관 M+],\n펀드매니저들은 매년 이맘때면 한 해의 운용 성과를 되돌아보고 내년을 전망하는데 ...


In [50]:
news_df.to_csv('news.csv', index=False)